In [1]:
import pandas as pd

import functions as f
import time
from comparing import ComparatorForDatasets
from comparing import CategoricalSimilarity
from similarity.DataFrameMetadataCreator import DataFrameMetadataCreator
from similarity.DataFrameMetadata import DataFrameMetadata
from collections import defaultdict

ModuleNotFoundError: No module named 'Types'

In [29]:
import gensim.downloader as api


wv = api.load('word2vec-google-news-300')


KeyboardInterrupt: 

In [ ]:
database, names = f.load__csv_files_from_folder("data")

In [ ]:
metadata : dict[str, DataFrameMetadata] = defaultdict()
for dataframe, name in zip(database, names):
    metadata[name] = DataFrameMetadataCreator(dataframe).create_column_embeddings().get_metadata()
    

In [14]:
from similarity.DataFrameMetadata import DataFrameMetadata
from collections import defaultdict
from itertools import compress

categorical_all =  list()
metadata_all = list()
metadata : dict[str, DataFrameMetadata] = defaultdict()
for dataframe, name in zip(database, names):
    metadata[name] = f.DataFrameMetadataCreator(dataframe).compute_correlation(0.5).create_column_embeddings().get_metadata()
    categorical_all.append(dataframe[list(compress(metadata[name].column_names, metadata[name].column_categorical))])
    metadata_all.append(metadata[name].categorical_metadata)

In [15]:
metadata_all[0].keys()


dict_keys(['reg_expiration', 'reg_state'])

In [16]:
import numpy as np


def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [23]:
def compute_similarity_score(similarity_matrix: defaultdict): ## todo test some other methods
    res = 0
    count = 0
    trashold = 0.7
    length = 0
    for i in similarity_matrix.values():
        avg = sum(i)/len(i)
        length = len(i)
        if max(i) > trashold:
            count += 1
        res += max(i)
    return count, res/len(similarity_matrix) * (count/len(similarity_matrix))

In [18]:
def get_ratio(count1: int, count2: int):
    if count1 > count2:
        return count1/count2
    else:
        return count2/count1

In [26]:

from comparing import SimilarityData, SimilarityStruct

start_time = time.perf_counter()
result = list()
for category_metadata in metadata_all:
    table_data = SimilarityData()
    for to_compare_metadata, to_compare_table_name in zip( metadata_all, names):
        if category_metadata != to_compare_metadata:
            for column_name, column in category_metadata.items():
                for column_compare_name, column_to_compare in to_compare_metadata.items():
                    simil_matrix = defaultdict(list)
                    category_to_compare_embed = column_to_compare.category_embedding
                    for category, name in zip(column.category_embedding, column.categories):
                        for category_to_compare in category_to_compare_embed:
                            simil_matrix[name].append(cosine(category, category_to_compare))     
                    count, score = compute_similarity_score(simil_matrix)
                    similarity = CategoricalSimilarity(
                        categories_ratio=get_ratio(column.count_categories, column_to_compare.count_categories),
                        count_similar=count,
                        similarity_score=score
                    )
                    similarity_struct = SimilarityStruct(categorical_similarity=similarity,
                      column_name=column_compare_name,
                      table_name=to_compare_table_name                                  )
                    table_data.similarity_columns[column_name].append(similarity_struct)
    result.append(table_data)
    print("Table done")
    
finish_time = time.perf_counter()
print("Program finished in {} seconds".format(finish_time-start_time))     

KeyboardInterrupt: 

In [ ]:
metadata_all[0]['model'].categories

In [ ]:
categorical_all[0]

In [ ]:
categorical_all[7]

In [ ]:
matrix = [[0.6, 1, 0.3, 0.4],
          [1, 0.4, 0.3, 0.4],
          [0.6, 0.2, 1, 0.4],
          ]
for i in matrix:
    print(sum(i), " ", max(i))